In [8]:
import constellate
import json
import pandas as pd
import pickle
pd.set_option("display.max_columns", None)
import os
import csv

In [2]:
dataset = constellate.dataset_reader("../data/large_files/jstor_v2023.jsonl")

In [3]:
type(dataset)

generator

In [4]:
data_columns = ['unigramCount', 'bigramCount', 'trigramCount', 'abstract', 'fullText']


In [5]:
path_trigrams = "../data/large_files/trigrams_raw/"
#os.makedirs(path_trigrams)
path_unigrams = "../data/large_files/unigrams_raw/"
#os.makedirs(path_unigrams)

In [7]:
%%time
metadata_list = []
dataset = constellate.dataset_reader("../data/large_files/jstor_v2023.jsonl")
for n, doc_dict in enumerate(dataset):
        doc_dict["id_kase"] = n
        trigrams = doc_dict["trigramCount"]
        with open(path_trigrams + "trigrams_{}.json".format(str(n)), "w") as f:
            json.dump(trigrams, f)
        unigrams = doc_dict["unigramCount"]
        with open(path_unigrams + "unigrams_{}.json".format(str(n)), "w") as f:
            json.dump(unigrams, f)
        for col in ['unigramCount', 'bigramCount', 'trigramCount', 'fullText']:
            try:
                del doc_dict[col]
            except:
                pass
        metadata_list.append(doc_dict)

CPU times: user 4min 49s, sys: 24.1 s, total: 5min 13s
Wall time: 5min 41s


In [92]:
metadata_df = pd.DataFrame(metadata_list)
len(metadata_df)

25000

In [93]:
metadata_df.to_json("../data/metadata_df.json")

In [94]:
ids_dict = dict(zip(metadata_df["id"], metadata_df["id_kase"]))

In [95]:
with open("../data/large_files/ids_dict.json", "w") as f:
    json.dump(ids_dict, f)

In [2]:
#dataset = constellate.dataset_reader("../data/large_files/jstor_v1.jsonl")

In [9]:
destpath = "../data/large_files/raw_docs/"
#os.makedirs(destpath)

In [12]:
%%time
dataset_list = []
for n, doc in enumerate(dataset):
    with open(destpath + "_{}.pickle".format(str(n)), "wb") as f:
        pickle.dump(doc, f)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [ ]:
dataset

In [13]:
%%time
dataset_list = []
for doc in dataset:
    dataset_list.append(doc)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 6.91 µs


In [4]:
len(dataset_list)

14103

In [5]:
# how does an individual document looks like?
dataset_list[2000]

{'creator': ['J. M. Flood'],
 'datePublished': '1945-09-01',
 'docSubType': 'research-article',
 'docType': 'article',
 'id': 'http://www.jstor.org/stable/30099597',
 'identifier': [{'name': 'issn', 'value': '00393495'},
  {'name': 'local_uuid', 'value': 'efc8a71f-2bc1-3a8b-b4cc-200d8fa8bc60'},
  {'name': 'local_doi', 'value': '10.2307/30099597'},
  {'name': 'journal_id', 'value': 'studiesirishrev'}],
 'isPartOf': 'Studies: An Irish Quarterly Review',
 'issueNumber': '135',
 'language': ['eng'],
 'outputFormat': ['unigram', 'bigram', 'trigram'],
 'pageCount': 10,
 'pageEnd': '396',
 'pageStart': '387',
 'pagination': 'pp. 387-396',
 'provider': 'jstor',
 'publicationYear': 1945,
 'publisher': 'Irish Province of the Society of Jesus',
 'sourceCategory': ['Humanities', 'Religion'],
 'tdmCategory': ['Religion - Theology', 'History - Historical methodology'],
 'title': 'Cardinal Newman and Dr. Charles Russell',
 'url': 'http://www.jstor.org/stable/30099597',
 'volumeNumber': '34',
 'wordCo

In [6]:
# transform it into a dataframe object
jstor_df = pd.DataFrame(dataset_list)

In [7]:
del dataset_list

In [9]:
jstor_df["id_kase"] = [n for n in range(len(jstor_df))]

In [8]:
data_columns = ['unigramCount', 'bigramCount', 'trigramCount', 'abstract', 'fullText']

In [10]:
for column in data_columns:
    dict_to_save = dict(zip(jstor_df["id_kase"], jstor_df[column]))
    pickle.dump(dict_to_save, open("../data/large_files/{0}_dict.pickle".format(column), "wb"))

In [11]:
jstor_df.drop(columns=data_columns, inplace=True)
jstor_df.head(5)

,creator,datePublished,docType,doi,id,identifier,isPartOf,issueNumber,keyphrase,language,outputFormat,pageCount,pageEnd,pageStart,pagination,provider,publicationYear,publisher,sequence,tdmCategory,title,url,volumeNumber,wordCount,docSubType,sourceCategory,subTitle,hasPartTitle,id_kase
0,[],1959-10-01,article,10.1177/001452465907100107,ark://27927/phx66812gq6,"[{'name': 'doi', 'value': '10.1177/00145246590...",Expository Times,1,"[omnipotent reigneth, god omnipotent, silence,...",[eng],"[unigram, bigram, trigram]",8.0,31,24,pp. 24-31,portico,1959,SAGE Publications,7.0,"[Religion - Theology, Religion - Spiritual bel...",In the Study,http://doi.org/10.1177/001452465907100107,71,7382,NaN,NaN,NaN,NaN,0
1,[Victor Paul Furnish],2009-01-01,chapter,10.1017/CBO9780511621321.003,ark://27927/pbd6fpf5fh,"[{'name': 'isbn', 'value': '9780511621321'}, {...",Jesus According to PaulJesus According to Paul,NaN,"[saying, books online, jesus tradition, pauls ...",[eng],"[unigram, bigram, trigram]",26.0,65,40,40-65,portico,2009,Cambridge University Press,5.0,"[History - Historical methodology, Religion - ...",3 Sayings of Jesus in Paul's Letters,https://doi.org/10.1017/CBO9780511621321.003,NaN,8577,NaN,NaN,NaN,NaN,1
2,[Leander E. Keck],2015-01-01,chapter,NaN,ark://27927/phw1kd8s300,[],Christ&#39;s First Theologian,NaN,"[pharisaism, sandmel genius, judaism, rabbinic...",[eng],"[unigram, bigram, trigram]",14.0,42,29,29-42,portico,2015,Baylor University Press,8.0,"[History - Historical methodology, Religion - ...",3. The Quest for Pauls Pharisaism,NaN,NaN,5354,NaN,NaN,NaN,NaN,2
3,[LeAnn Snow Flesher],2009-02-01,article,10.1177/003463730910600105,ark://27927/phx64fptrwj,"[{'name': 'doi', 'value': '10.1177/00346373091...",Review & Expositor: An International Baptist J...,1,"[scofield, premillennial, premillennial dispen...",[eng],"[unigram, bigram, trigram]",11.0,45,35,pp. 35-45,portico,2009,SAGE Publications,5.0,"[Religion - Theology, Religion - Spiritual bel...",The Historical Development of Premillennial Di...,http://doi.org/10.1177/003463730910600105,106,3614,NaN,NaN,NaN,NaN,3
4,[A. Daunton-Fear],1995-07-01,article,10.1177/0040571X9509800404,ark://27927/phx64k1x5c2,"[{'name': 'doi', 'value': '10.1177/0040571X950...",Theology,784,"[baptism, holy spirit, communion, infant bapti...",[eng],"[unigram, bigram, trigram]",10.0,282,273,273-282,portico,1995,SAGE Publications,4.0,[Religion - Spiritual belief systems],Resisting the Tide Christian Initiation and Co...,http://doi.org/10.1177/0040571X9509800404,98,4323,NaN,NaN,NaN,NaN,4


In [12]:
len(jstor_df)

14103

In [13]:
%%time
jstor_df.to_feather("../data/large_files/jstor_df_v1.feather")

CPU times: user 221 ms, sys: 201 ms, total: 423 ms
Wall time: 888 ms
